In [3]:
import pandas as pd 
import os
import glob
import pandas as pd
import numpy as np


In [22]:
# Function to load and process directory log file
def load_proc_dir_log_file(path):
    folder = glob.glob(os.path.join(path, '*/*csv'))
    full_dict = []  # Use more descriptive variable names
    
    for file_path in folder:
        
        df = pd.read_csv(file_path)
        region = os.path.basename(os.path.dirname(file_path)) 
        # region = file_path.split('/')[-3]
        batch = os.path.basename(file_path).split('_')[0]
        data_len = len(df)
        
        # Dictionary creation simplified
        full_dict.append({
            'path': file_path,
            'region': region,
            'batch': batch,
            'len': data_len,
            'memory': 'norm'
        })
    
    log_df = pd.DataFrame(full_dict)
    return log_df


def test_age_outputs(df):
    # check that only unknowns have diff values 
    def check_cols_same(col):
        if not df[df[f'localfill_{col}']!=df[f'globalfill_{col}']].empty:   
            print(f'Error: Local and global {col} are different for known ages')
            raise Exception('Error: Local and global are different for known ages')

    df = df[df['count_unknown_age']==0].copy() 
    for col in ['modal_age', 'median_age', 'max_age', 'min_age', 'range_age', 'iqr_age']:
        check_cols_same(col)

    print('Checks complete')


# Function to process and filter final DataFrame
def proc_fin_df(fin_df):
    test_age_outputs(fin_df)
   
    return fin_df 


In [14]:
log = load_proc_dir_log_file('/Volumes/T9/Data_downloads/new-data-outputs/age') 

In [16]:
log.sort_values(['region', 'batch'])

,path,region,batch,len,memory
42,/Volumes/T9/Data_downloads/new-data-outputs/ag...,EE,0,10000,norm
40,/Volumes/T9/Data_downloads/new-data-outputs/ag...,EE,1,10000,norm
38,/Volumes/T9/Data_downloads/new-data-outputs/ag...,EE,10,10000,norm
32,/Volumes/T9/Data_downloads/new-data-outputs/ag...,EE,11,10000,norm
36,/Volumes/T9/Data_downloads/new-data-outputs/ag...,EE,12,10000,norm
...,...,...,...,...,...
88,/Volumes/T9/Data_downloads/new-data-outputs/ag...,YH,5,7650,norm
78,/Volumes/T9/Data_downloads/new-data-outputs/ag...,YH,6,7800,norm
81,/Volumes/T9/Data_downloads/new-data-outputs/ag...,YH,7,6160,norm
75,/Volumes/T9/Data_downloads/new-data-outputs/ag...,YH,8,7940,norm


In [32]:
fin = [] 
for x in log.path:
    
    f= pd.read_csv(x)

    fin.append(f)
fin_df = pd.concat(fin)

res_df = proc_fin_df(fin_df)
# res_df.dropna(axis=0, inplace=True )
# res_df.dropna(axis=1, how='all' , inplace=True )

Checks complete


In [35]:
data = res_df[~res_df['globalfill_median_age'].isna()].copy() 

data[data['localfill_median_age']!= data['globalfill_median_age']]

,postcode,count_unknown_age,localfill_median_age,localfill_modal_age,localfill_min_age,localfill_max_age,localfill_range_age,localfill_distinct_ages,localfill_iqr_age,globalfill_median_age,globalfill_modal_age,globalfill_min_age,globalfill_max_age,globalfill_range_age,globalfill_distinct_ages,globalfill_iqr_age
69,EX31 3GG,1.0,1990-1999,1990-1999,Pre 1919,Post 1999,6.0,5.0,4.25,1980-1989,1990-1999,Pre 1919,Post 1999,6.0,6.0,4.25
90,EX31 3PE,1.0,Pre 1919,Pre 1919,Pre 1919,Post 1999,6.0,3.0,3.75,1945-1959,Pre 1919,Pre 1919,Post 1999,6.0,4.0,4.50
838,EX33 1BX,54.0,1980-1989,Unknown date,1960-1979,Post 1999,3.0,3.0,1.50,1960-1979,1960-1979,1919-1944,Post 1999,5.0,4.0,0.00
859,EX34 9SF,1.0,1919-1944,1919-1944,Pre 1919,Post 1999,6.0,6.0,4.00,1945-1959,1919-1944,Pre 1919,Post 1999,6.0,6.0,4.50
1520,PL26 7AX,7.0,1919-1944,1919-1944,Pre 1919,Post 1999,6.0,5.0,4.25,1960-1979,1919-1944,Pre 1919,Post 1999,6.0,5.0,4.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5786,NE65 7XS,1.0,1919-1944,1919-1944,1919-1944,1919-1944,0.0,1.0,0.00,1980-1989,1919-1944,1919-1944,Post 1999,5.0,2.0,2.50
4160,NE46 4RP,1.0,1919-1944,1919-1944,Pre 1919,1990-1999,5.0,3.0,1.25,1945-1959,1919-1944,Pre 1919,1990-1999,5.0,4.0,2.75
5232,NE43 7LN,1.0,1919-1944,1919-1944,1919-1944,1919-1944,0.0,1.0,0.00,1945-1959,1919-1944,1919-1944,1960-1979,2.0,2.0,1.00
368,TS21 2BT,1.0,Pre 1919,Pre 1919,Pre 1919,1990-1999,5.0,3.0,2.00,1919-1944,Pre 1919,Pre 1919,1990-1999,5.0,3.0,2.75


: 

In [31]:
res_df.localfill_median_age.isna().sum(), res_df.globalfill_median_age.isna().sum()

(0, 0)